In [565]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/analytics-olympiad-2022/AnalyticsOlympiad2022Data/submission.csv
/kaggle/input/analytics-olympiad-2022/AnalyticsOlympiad2022Data/train.csv
/kaggle/input/analytics-olympiad-2022/AnalyticsOlympiad2022Data/test.csv


In [566]:
!pip3 install pyzipcode

In [567]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, ExtraTreesClassifier
from sklearn.metrics import log_loss
from xgboost import XGBClassifier, XGBRegressor
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB, CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2

In [568]:
from lightgbm import LGBMClassifier
from pyzipcode import ZipCodeDatabase

In [569]:
tr = pd.read_csv("../input/analytics-olympiad-2022/AnalyticsOlympiad2022Data/train.csv")
te = pd.read_csv("../input/analytics-olympiad-2022/AnalyticsOlympiad2022Data/test.csv")
sample = pd.read_csv("../input/analytics-olympiad-2022/AnalyticsOlympiad2022Data/submission.csv")

In [570]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105000 entries, 0 to 104999
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   105000 non-null  int64  
 1   AGE                  105000 non-null  object 
 2   GENDER               105000 non-null  object 
 3   DRIVING_EXPERIENCE   105000 non-null  object 
 4   EDUCATION            105000 non-null  object 
 5   INCOME               105000 non-null  object 
 6   CREDIT_SCORE         105000 non-null  float64
 7   VEHICLE_OWNERSHIP    105000 non-null  float64
 8   VEHICLE_YEAR         105000 non-null  object 
 9   MARRIED              105000 non-null  float64
 10  CHILDREN             105000 non-null  float64
 11  POSTAL_CODE          105000 non-null  int64  
 12  ANNUAL_MILEAGE       105000 non-null  float64
 13  SPEEDING_VIOLATIONS  105000 non-null  int64  
 14  DUIS                 105000 non-null  int64  
 15  PAST_ACCIDENTS   

In [571]:
tr

,ID,AGE,GENDER,DRIVING_EXPERIENCE,EDUCATION,INCOME,CREDIT_SCORE,VEHICLE_OWNERSHIP,VEHICLE_YEAR,MARRIED,CHILDREN,POSTAL_CODE,ANNUAL_MILEAGE,SPEEDING_VIOLATIONS,DUIS,PAST_ACCIDENTS,OUTCOME,TYPE_OF_VEHICLE
0,816393,40-64,female,20-29y,university,middle class,0.638050,0.0,after 2015,0.0,0.0,37379,11000.0,0,0,0,0.0,Sports Car
1,251762,26-39,male,20-29y,high school,middle class,0.475741,1.0,before 2015,1.0,0.0,10238,9000.0,0,0,0,1.0,HatchBack
2,481952,40-64,male,20-29y,none,middle class,0.839817,1.0,before 2015,1.0,1.0,10238,12000.0,0,0,0,1.0,Sedan
3,3506,40-64,male,20-29y,high school,upper class,0.682527,1.0,before 2015,0.0,1.0,92099,6000.0,1,0,0,1.0,Sedan
4,498013,40-64,female,20-29y,none,working class,0.572184,1.0,after 2015,1.0,1.0,32122,15000.0,0,0,1,0.0,Sedan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104995,19557,65+,female,0-9y,high school,upper class,0.672255,1.0,before 2015,0.0,0.0,31885,9000.0,2,0,0,0.0,SUV
104996,222459,40-64,female,0-9y,none,working class,0.514193,1.0,after 2015,1.0,0.0,32277,10000.0,0,0,0,1.0,Sedan
104997,83089,16-25,male,10-19y,none,upper class,0.461942,1.0,after 2015,1.0,0.0,34800,11000.0,0,0,0,1.0,Sedan
104998,984879,40-64,female,20-29y,high school,working class,0.483571,1.0,before 2015,0.0,0.0,10238,10000.0,0,0,0,0.0,HatchBack


In [572]:
# # le = LabelEncoder()
# tr['CREDIT_SCORE'] *= 1000 
# te['CREDIT_SCORE'] *= 1000

# tr['Poor_Credit_Score'] = np.where(((tr['CREDIT_SCORE'] >= 300) & (tr['CREDIT_SCORE']  < 500)), 1, 0)
# te['Poor_Credit_Score'] = np.where(((te['CREDIT_SCORE'] >= 300) & (te['CREDIT_SCORE']  < 500)), 1, 0)

# tr['Avg_Credit_Score'] = np.where(((tr['CREDIT_SCORE'] >= 500) & (tr['CREDIT_SCORE'] < 650)), 1, 0)
# te['Avg_Credit_Score'] = np.where(((te['CREDIT_SCORE'] >= 500) & (te['CREDIT_SCORE'] < 650)), 1, 0)

# tr['Good_Credit_Score'] = np.where(((tr['CREDIT_SCORE'] >= 650) & (tr['CREDIT_SCORE'] < 750)), 1, 0)
# te['Good_Credit_Score'] = np.where(((te['CREDIT_SCORE'] >= 650) & (te['CREDIT_SCORE'] < 750)), 1, 0)

# tr['Excellent_Credit_Score'] = np.where(((tr['CREDIT_SCORE'] >= 750) & (tr['CREDIT_SCORE'] < 900)), 1, 0)
# te['Excellent_Credit_Score'] = np.where(((te['CREDIT_SCORE'] >= 750) & (te['CREDIT_SCORE'] < 900)), 1, 0)

# tr['SuperExcellent_Credit_Score'] = np.where((tr['CREDIT_SCORE'] >= 900), 1, 0)
# te['SuperExcellent_Credit_Score'] = np.where((te['CREDIT_SCORE'] >= 900), 1, 0)


In [573]:
# tr['Standard_Annual_Mileage'] = np.where(((tr['ANNUAL_MILEAGE'] >= 8000) & (tr['ANNUAL_MILEAGE'] <= 10000)),1, 0)
# te['Standard_Annual_Mileage'] = np.where(((te['ANNUAL_MILEAGE'] >= 8000) & (te['ANNUAL_MILEAGE'] <= 10000)),1, 0)

# tr['Low_Annual_Mileage'] = np.where((tr['ANNUAL_MILEAGE'] < 8000), 1, 0)
# te['Low_Annual_Mileage'] = np.where((te['ANNUAL_MILEAGE'] < 8000), 1, 0)

# tr['High_Annual_Mileage'] = np.where((tr['ANNUAL_MILEAGE'] > 10000), 1, 0)
# te['High_Annual_Mileage'] = np.where((te['ANNUAL_MILEAGE'] > 10000), 1, 0)

In [574]:
# tr.drop(['CREDIT_SCORE', 'ANNUAL_MILEAGE'], axis = 1, inplace =True)
# te.drop(['CREDIT_SCORE', 'ANNUAL_MILEAGE'], axis = 1, inplace =True)


In [575]:
tr['VEHICLE_OWNERSHIP'] = tr['VEHICLE_OWNERSHIP'].astype('object')
te['VEHICLE_OWNERSHIP'] = te['VEHICLE_OWNERSHIP'].astype('object')

tr['CHILDREN'] = tr['CHILDREN'].astype('object')
te['CHILDREN'] = te['CHILDREN'].astype('object')

tr['MARRIED'] = tr['MARRIED'].astype('object')
te['MARRIED'] = te['MARRIED'].astype('object')

tr['OUTCOME'] = tr['OUTCOME'].astype('int')

In [576]:
cat_feat = tr.select_dtypes(include = ["object"]).columns.to_list()
cat_feat

['AGE',
 'GENDER',
 'DRIVING_EXPERIENCE',
 'EDUCATION',
 'INCOME',
 'VEHICLE_OWNERSHIP',
 'VEHICLE_YEAR',
 'MARRIED',
 'CHILDREN',
 'TYPE_OF_VEHICLE']

In [577]:
tr = pd.get_dummies(tr, columns = cat_feat, drop_first = True)
te = pd.get_dummies(te, columns = cat_feat, drop_first = True)

In [578]:
tr.describe()

,ID,CREDIT_SCORE,POSTAL_CODE,ANNUAL_MILEAGE,SPEEDING_VIOLATIONS,DUIS,PAST_ACCIDENTS,OUTCOME,AGE_26-39,AGE_40-64,...,INCOME_poverty,INCOME_upper class,INCOME_working class,VEHICLE_OWNERSHIP_1.0,VEHICLE_YEAR_before 2015,MARRIED_1.0,CHILDREN_1.0,TYPE_OF_VEHICLE_SUV,TYPE_OF_VEHICLE_Sedan,TYPE_OF_VEHICLE_Sports Car
count,105000.000000,105000.000000,105000.000000,105000.000000,105000.000000,105000.000000,105000.000000,105000.000000,105000.000000,105000.000000,...,105000.000000,105000.000000,105000.000000,105000.000000,105000.000000,105000.000000,105000.000000,105000.000000,105000.00000,105000.000000
mean,394931.295905,0.602173,18045.439267,11061.228571,0.675676,0.129733,0.549314,0.422648,0.190648,0.321105,...,0.138448,0.488295,0.232895,0.827038,0.547724,0.584143,0.520000,0.165600,0.26781,0.329448
std,279694.106053,0.138045,16709.040449,2972.355482,1.383678,0.589714,1.402809,0.493983,0.392814,0.466903,...,0.345371,0.499865,0.422678,0.378216,0.497720,0.492871,0.499602,0.371723,0.44282,0.470015
min,101.000000,0.066880,10238.000000,2000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,156351.750000,0.514876,10238.000000,9000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,354679.500000,0.601112,10238.000000,11000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.00000,0.000000
75%,598602.250000,0.703216,22957.750000,13000.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,...,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.00000,1.000000
max,999976.000000,0.954075,92101.000000,21000.000000,20.000000,6.000000,15.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000


In [579]:
# tr['AGE_min'] = np.where(tr['AGE'] == '16-25', 16, np.where(tr['AGE'] == '26-39', 26, np.where(tr['AGE'] == '40-64', 40, np.where(tr['AGE'] == '65+', 65, 40))))
# tr['AGE_max'] = np.where(tr['AGE'] == '16-25', 25, np.where(tr['AGE'] == '26-39', 39, np.where(tr['AGE'] == '40-64', 64, np.where(tr['AGE'] == '65+', 100, 64))))

# te['AGE_min'] = np.where(te['AGE'] == '16-25', 16, np.where(te['AGE'] == '26-39', 26, np.where(te['AGE'] == '40-64', 40, np.where(te['AGE'] == '65+', 65, 40))))
# te['AGE_max'] = np.where(te['AGE'] == '16-25', 25, np.where(te['AGE'] == '26-39', 39, np.where(te['AGE'] == '40-64', 64, np.where(te['AGE'] == '65+', 100, 64))))

In [580]:
# tr['AGE_median'] = np.where(tr['AGE'] == '16-25', (16+25)/2, np.where(tr['AGE'] == '26-39', (26+39)/2, np.where(tr['AGE'] == '40-64', (40+64)/2, np.where(tr['AGE'] == '65+', (65+100)/2, (40+64)/2))))
# te['AGE_median'] = np.where(te['AGE'] == '16-25', (16+25)/2, np.where(te['AGE'] == '26-39', (26+39)/2, np.where(te['AGE'] == '40-64', (40+64)/2, np.where(te['AGE'] == '65+', (65+100)/2, (40+64)/2))))

In [581]:
# tr['DRIVING_EXPERIENCE_min'] = np.where(tr['DRIVING_EXPERIENCE'] == '20-29y', 20, np.where(tr['DRIVING_EXPERIENCE'] == '0-9y', 0, np.where(tr['DRIVING_EXPERIENCE'] == '10-19y', 10, np.where(tr['DRIVING_EXPERIENCE'] == '30y+', 30, 20))))
# tr['DRIVING_EXPERIENCE_max'] = np.where(tr['DRIVING_EXPERIENCE'] == '20-29y', 29, np.where(tr['DRIVING_EXPERIENCE'] == '0-9y', 9, np.where(tr['DRIVING_EXPERIENCE'] == '10-19y', 19, np.where(tr['DRIVING_EXPERIENCE'] == '30y+', 100, 29))))

# te['DRIVING_EXPERIENCE_min'] = np.where(te['DRIVING_EXPERIENCE'] == '20-29y', 20, np.where(te['DRIVING_EXPERIENCE'] == '0-9y', 0, np.where(te['DRIVING_EXPERIENCE'] == '10-19y', 10, np.where(te['DRIVING_EXPERIENCE'] == '30y+', 30, 20))))
# te['DRIVING_EXPERIENCE_max'] = np.where(te['DRIVING_EXPERIENCE'] == '20-29y', 29, np.where(te['DRIVING_EXPERIENCE'] == '0-9y', 9, np.where(te['DRIVING_EXPERIENCE'] == '10-19y', 19, np.where(te['DRIVING_EXPERIENCE'] == '30y+', 100, 29))))

In [582]:
# tr['DRIVING_EXPERIENCE_median'] = np.where(tr['DRIVING_EXPERIENCE'] == '20-29y', (20+29)/2, np.where(tr['DRIVING_EXPERIENCE'] == '0-9y', (0+9)/2, np.where(tr['DRIVING_EXPERIENCE'] == '10-19y', (10+19)/2, np.where(tr['DRIVING_EXPERIENCE'] == '30y+', (30+100)/2, (20+29)/2))))
# te['DRIVING_EXPERIENCE_median'] = np.where(te['DRIVING_EXPERIENCE'] == '20-29y', (20+29)/2, np.where(te['DRIVING_EXPERIENCE'] == '0-9y', (0+9)/2, np.where(te['DRIVING_EXPERIENCE'] == '10-19y', (10+19)/2, np.where(te['DRIVING_EXPERIENCE'] == '30y+', (30+100)/2, (20+29)/2))))

In [583]:
# tr['car_learning_time'] = tr['DRIVING_EXPERIENCE_median'] - tr['AGE_median']
# te['car_learning_time'] = te['DRIVING_EXPERIENCE_median'] - te['AGE_median']

In [584]:
# tr['INCOME_cat'] = np.where(tr['INCOME'] == 'poverty', 0, np.where(tr['INCOME'] == 'working class', 1, np.where(tr['INCOME'] == 'middle class', 2, np.where(tr['INCOME'] == 'upper class', 3, 3))))
# te['INCOME_cat'] = np.where(te['INCOME'] == 'poverty', 0, np.where(te['INCOME'] == 'working class', 1, np.where(te['INCOME'] == 'middle class', 2, np.where(te['INCOME'] == 'upper class', 3, 3))))

# tr['EDU_cat'] = np.where(tr['EDUCATION'] == 'none', 0, np.where(tr['EDUCATION'] == 'high school', 1, 2))
# te['EDU_cat'] = np.where(te['EDUCATION'] == 'none', 0, np.where(te['EDUCATION'] == 'high school', 1, 2))

In [585]:
# tr['GENDER_cat'] = np.where(tr['GENDER'] == 'male', 1, 0)
# te['GENDER_cat'] = np.where(te['GENDER'] == 'male', 1, 0)

# tr['VEHICLE_YEAR_cat'] = np.where(tr['VEHICLE_YEAR'] == 'before 2015', 1, 0)
# te['VEHICLE_YEAR_cat'] = np.where(te['VEHICLE_YEAR'] == 'before 2015', 1, 0)

# tr['TYPE_OF_VEHICLE'] = np.where(tr['TYPE_OF_VEHICLE'] == 'HatchBack', 0, np.where(tr['TYPE_OF_VEHICLE'] == 'Sedan', 1, np.where(tr['TYPE_OF_VEHICLE'] == 'SUV', 2, np.where(tr['TYPE_OF_VEHICLE'] == 'Sports Car', 3, 3))))
# te['TYPE_OF_VEHICLE'] = np.where(te['TYPE_OF_VEHICLE'] == 'HatchBack', 0, np.where(te['TYPE_OF_VEHICLE'] == 'Sedan', 1, np.where(te['TYPE_OF_VEHICLE'] == 'SUV', 2, np.where(te['TYPE_OF_VEHICLE'] == 'Sports Car', 3, 3))))

# tr['VEHICLE_OWNERSHIP'] = le.fit_transform(tr['VEHICLE_OWNERSHIP'])
# te['VEHICLE_OWNERSHIP'] = le.fit_transform(te['VEHICLE_OWNERSHIP'])

# tr['MARRIED'] = le.fit_transform(tr['MARRIED'])
# te['MARRIED'] = le.fit_transform(te['MARRIED'])

# tr['CHILDREN'] = le.fit_transform(tr['CHILDREN'])
# te['CHILDREN'] = le.fit_transform(te['CHILDREN'])

In [586]:
# tr[cat_feat]

In [587]:
# le.fit_transform(tr[cat_feat])
# le.fit_transform(te[cat_feat])
# for i in cat_feat:
#     tr[i] = le.fit_transform(tr[i])
#     te[i] = le.fit_transform(te[i])

In [588]:
# tr.drop(cat_feat, axis = 1, inplace = True)
# te.drop(cat_feat, axis = 1, inplace = True)

In [589]:
# X = tr.drop(['OUTCOME'], axis = 1)
# y = tr['OUTCOME']
# skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
# # skf = KFold(n_splits = 10, shuffle = True, random_state = 42)


# enc_val_df = []
# i = 1
# for train_index, test_index in skf.split(X, y):
#     print("{} of KFold {}".format(i, skf.n_splits))
#     xtr,xvl = X.loc[train_index],X.loc[test_index]
#     ytr,yvl = y.loc[train_index],y.loc[test_index]
#     df_train = pd.concat([xtr, ytr], axis = 1)
#     df_valid = pd.concat([xvl, yvl], axis = 1)
#     for j in cat_feat:
#         mapping_dict = dict(df_train.groupby(j)["OUTCOME"].mean())
#         df_valid.loc[:, j + '_tar_enc'] = df_valid[j].map(mapping_dict)
#     enc_val_df.append(df_valid)
#     i+=1
# tr = pd.concat(enc_val_df, axis = 0)


In [590]:
# enc_val_df

In [591]:
tr

,ID,CREDIT_SCORE,POSTAL_CODE,ANNUAL_MILEAGE,SPEEDING_VIOLATIONS,DUIS,PAST_ACCIDENTS,OUTCOME,AGE_26-39,AGE_40-64,...,INCOME_poverty,INCOME_upper class,INCOME_working class,VEHICLE_OWNERSHIP_1.0,VEHICLE_YEAR_before 2015,MARRIED_1.0,CHILDREN_1.0,TYPE_OF_VEHICLE_SUV,TYPE_OF_VEHICLE_Sedan,TYPE_OF_VEHICLE_Sports Car
0,816393,0.638050,37379,11000.0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,251762,0.475741,10238,9000.0,0,0,0,1,1,0,...,0,0,0,1,1,1,0,0,0,0
2,481952,0.839817,10238,12000.0,0,0,0,1,0,1,...,0,0,0,1,1,1,1,0,1,0
3,3506,0.682527,92099,6000.0,1,0,0,1,0,1,...,0,1,0,1,1,0,1,0,1,0
4,498013,0.572184,32122,15000.0,0,0,1,0,0,1,...,0,0,1,1,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104995,19557,0.672255,31885,9000.0,2,0,0,0,0,0,...,0,1,0,1,1,0,0,1,0,0
104996,222459,0.514193,32277,10000.0,0,0,0,1,0,1,...,0,0,1,1,0,1,0,0,1,0
104997,83089,0.461942,34800,11000.0,0,0,0,1,0,0,...,0,1,0,1,0,1,0,0,1,0
104998,984879,0.483571,10238,10000.0,0,0,0,0,0,1,...,0,0,1,1,1,0,0,0,0,0


In [592]:
# for i in cat_feat:
#     mapping_dict = dict(tr.groupby(i)['OUTCOME'].mean())
#     te.loc[:,i+'_tar_enc'] = te[i].map(mapping_dict)

In [593]:
# te

In [594]:
# hehe

In [595]:
tr['POSTAL_CODE'] = tr['POSTAL_CODE'].astype('str')
te['POSTAL_CODE'] = te['POSTAL_CODE'].astype('str')

# full = pd.concat([tr[['POSTAL_CODE']], te[['POSTAL_CODE']]], axis = 0)

In [596]:
# full['POSTAL_CODE'] = full['POSTAL_CODE'].astype('str')
tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105000 entries, 0 to 104999
Data columns (total 27 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   ID                          105000 non-null  int64  
 1   CREDIT_SCORE                105000 non-null  float64
 2   POSTAL_CODE                 105000 non-null  object 
 3   ANNUAL_MILEAGE              105000 non-null  float64
 4   SPEEDING_VIOLATIONS         105000 non-null  int64  
 5   DUIS                        105000 non-null  int64  
 6   PAST_ACCIDENTS              105000 non-null  int64  
 7   OUTCOME                     105000 non-null  int64  
 8   AGE_26-39                   105000 non-null  uint8  
 9   AGE_40-64                   105000 non-null  uint8  
 10  AGE_65+                     105000 non-null  uint8  
 11  GENDER_male                 105000 non-null  uint8  
 12  DRIVING_EXPERIENCE_10-19y   105000 non-null  uint8  
 13  DRIVING_EXPERI

In [597]:
# full.info()
tr['POSTAL_CODE_1st_digit']  = tr['POSTAL_CODE'].apply(lambda x: x[0])
te['POSTAL_CODE_1st_digit']  = te['POSTAL_CODE'].apply(lambda x: x[0])

tr['POSTAL_CODE_2nd_digit']  = tr['POSTAL_CODE'].apply(lambda x: x[1])
te['POSTAL_CODE_2nd_digit']  = te['POSTAL_CODE'].apply(lambda x: x[1])

tr['POSTAL_CODE_3rd_digit']  = tr['POSTAL_CODE'].apply(lambda x: x[2])
te['POSTAL_CODE_3rd_digit']  = te['POSTAL_CODE'].apply(lambda x: x[2])

tr['POSTAL_CODE_4th_digit']  = tr['POSTAL_CODE'].apply(lambda x: x[3])
te['POSTAL_CODE_4th_digit']  = te['POSTAL_CODE'].apply(lambda x: x[3])

tr['POSTAL_CODE_5th_digit']  = tr['POSTAL_CODE'].apply(lambda x: x[4])
te['POSTAL_CODE_5th_digit']  = te['POSTAL_CODE'].apply(lambda x: x[4])

# tr['POSTAL_CODE_2nd_3rd_digit']  = tr['POSTAL_CODE'].apply(lambda x: x[1:3])
# te['POSTAL_CODE_2nd_3rd_digit']  = te['POSTAL_CODE'].apply(lambda x: x[1:3])

# tr['POSTAL_CODE_4th_5th_digit'] = tr['POSTAL_CODE'].apply(lambda x: x[3:5])
# te['POSTAL_CODE_4th_5th_digit'] = te['POSTAL_CODE'].apply(lambda x: x[3:5])

In [598]:
# tr['POSTAL_CODE_1st_digit']

In [599]:
# tr['POSTAL_CODE_2nd_3rd_digit'] 

In [600]:
# for i in range(tr.shape[0]):
#     for j in range(5):
#         for k in range(10):
#             if int(tr['POSTAL_CODE'][i][j]) == k:
#                 tr.loc[i, 'POSTAL_CODE_' + str(j+1) + 'digit' + str(k)] = 1
#             else:
#                 tr.loc[i, 'POSTAL_CODE_' + str(j+1) + 'digit' + str(k)] = 0

In [601]:
# for i in range(te.shape[0]):
#     for j in range(5):
#         for k in range(10):
#             if int(te['POSTAL_CODE'][i][j]) == k:
#                 te.loc[i, 'POSTAL_CODE_' + str(j+1) + 'digit' + str(k)] = 1
#             else:
#                 te.loc[i, 'POSTAL_CODE_' + str(j+1) + 'digit' + str(k)] = 0

In [602]:
# def postalcodeEncodingTopK(K: intfor i in range(te.shape[0]):
#     for j in range(5):
#         for k in range(10):
#             if int(te['POSTAL_CODE'][i][j]) == k:
#                 te.loc[i, 'POSTAL_CODE_' + str(j+1) + 'digit' + str(k)] = 1
#             else:
#                 te.loc[i, 'POSTAL_CODE_' + str(j+1) + 'digit' + str(k)] = 0, df):
    
#     topKPostalCodes = df['POSTAL_CODE'].value_counts().index.values[:K].tolist()
    
#     for i in topKPostalCodes:
#         df['POSTAL_CODE_' + str(i)] = np.where(df['POSTAL_CODE'] == i, 1, 0)
    
#     return topKPostalCodes, df

# def postalcodeDataFrame(topKPostalCodes: list, df_from, df_to, df_to_start_shape, df_to_end_shape):
#     for i in topKPostalCodes:
#         df_to['POSTAL_CODE_' + str(i)] = df_from['POSTAL_CODE_' + str(i)].iloc[df_to_start_shape:df_to_end_shape] 
#     return df_to

In [603]:
# K = 10
# topKPostalCodes, df = postalcodeEncodingTopK(K, full)
# tr, te = postalcodeDataFrame(topKPostalCodes, df, tr, 0, tr.shape[0]), postalcodeDataFrame(topKPostalCodes, df, te, tr.shape[0], full.shape[0])

In [604]:
# full = pd.concat([tr[['POSTAL_CODE']], te[['POSTAL_CODE']]], axis = 0)
# full = pd.get_dummies(full, columns = ['POSTAL_CODE'], drop_first = True)
# full

In [605]:
# train = pd.concat([tr, full.iloc[0:tr.shape[0],]], axis = 1)
# train = tr.copy()

In [606]:
# pca = PCA()
# # the number of components = 784
# pca.n_components = 2

# train = StandardScaler().fit_transform(train)

# # fit transform sample data using pca 
# pca_data = pca.fit_transform(train)


# #calculating percentage of variance explained in the data
# percentage_var_explained = pca.explained_variance_/np.sum(pca.explained_variance_)

# #cumulative sum of the percentage_var_explained
# cum_var_explained = np.cumsum(percentage_var_explained)

# # Plot the PCA spectrum
# plt.figure(1, figsize=(6, 4))

# plt.clf()
# plt.plot(cum_var_explained, linewidth = 2)
# plt.axis('tight')
# plt.grid()
# plt.xlabel('n_components')
# plt.ylabel('cumulative_explained_variance')
# plt.show()

In [607]:
# heh

In [608]:
# tr = pd.concat([tr, full.iloc[0:tr.shape[0],]], axis = 1)
# te = pd.concat([te, full.iloc[tr.shape[0]: full.shape[0], ]], axis = 1)

In [609]:
# zcdb = ZipCodeDatabase()

# tr['Postal_Code_Lat'] = tr['POSTAL_CODE'].apply(lambda x: zcdb[x].latitude)
# te['Postal_Code_Lat'] = te['POSTAL_CODE'].apply(lambda x: zcdb[x].latitude)

# tr['Postal_Code_Long'] = tr['POSTAL_CODE'].apply(lambda x: zcdb[x].longitude)
# te['Postal_Code_Long'] = te['POSTAL_CODE'].apply(lambda x: zcdb[x].longitude)

# zipcode = zcdb[54115]
# zipcode.zip
# u'54115'
# zipcode.city
# u'De Pere'
# zipcode.state
# u'WI'
# zipcode.longitude
# -88.078959999999995
# zipcode.latitude
# 44.42042
# zipcode.timezone
# -6

In [610]:
# heh

In [611]:
tr.drop(['POSTAL_CODE'], axis = 1, inplace = True)
te.drop(['POSTAL_CODE'], axis = 1, inplace = True)

In [612]:
tr.drop(['ID'], axis = 1, inplace = True)
te.drop(['ID'], axis = 1, inplace = True)

In [613]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105000 entries, 0 to 104999
Data columns (total 30 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   CREDIT_SCORE                105000 non-null  float64
 1   ANNUAL_MILEAGE              105000 non-null  float64
 2   SPEEDING_VIOLATIONS         105000 non-null  int64  
 3   DUIS                        105000 non-null  int64  
 4   PAST_ACCIDENTS              105000 non-null  int64  
 5   OUTCOME                     105000 non-null  int64  
 6   AGE_26-39                   105000 non-null  uint8  
 7   AGE_40-64                   105000 non-null  uint8  
 8   AGE_65+                     105000 non-null  uint8  
 9   GENDER_male                 105000 non-null  uint8  
 10  DRIVING_EXPERIENCE_10-19y   105000 non-null  uint8  
 11  DRIVING_EXPERIENCE_20-29y   105000 non-null  uint8  
 12  DRIVING_EXPERIENCE_30y+     105000 non-null  uint8  
 13  EDUCATION_none

In [614]:
cat_feat_new = tr.select_dtypes(include = 'object').columns.to_list()
cat_feat_new

['POSTAL_CODE_1st_digit',
 'POSTAL_CODE_2nd_digit',
 'POSTAL_CODE_3rd_digit',
 'POSTAL_CODE_4th_digit',
 'POSTAL_CODE_5th_digit']

In [615]:
tr = pd.get_dummies(tr, columns = cat_feat_new, drop_first = True)
te = pd.get_dummies(te, columns = cat_feat_new, drop_first = True)

In [616]:
# tr.drop(cat_feat, axis = 1, inplace = True)
# te.drop(cat_feat, axis = 1, inplace = True)

In [617]:
tr.columns

Index(['CREDIT_SCORE', 'ANNUAL_MILEAGE', 'SPEEDING_VIOLATIONS', 'DUIS',
       'PAST_ACCIDENTS', 'OUTCOME', 'AGE_26-39', 'AGE_40-64', 'AGE_65+',
       'GENDER_male', 'DRIVING_EXPERIENCE_10-19y', 'DRIVING_EXPERIENCE_20-29y',
       'DRIVING_EXPERIENCE_30y+', 'EDUCATION_none', 'EDUCATION_university',
       'INCOME_poverty', 'INCOME_upper class', 'INCOME_working class',
       'VEHICLE_OWNERSHIP_1.0', 'VEHICLE_YEAR_before 2015', 'MARRIED_1.0',
       'CHILDREN_1.0', 'TYPE_OF_VEHICLE_SUV', 'TYPE_OF_VEHICLE_Sedan',
       'TYPE_OF_VEHICLE_Sports Car', 'POSTAL_CODE_1st_digit_2',
       'POSTAL_CODE_1st_digit_3', 'POSTAL_CODE_1st_digit_4',
       'POSTAL_CODE_1st_digit_8', 'POSTAL_CODE_1st_digit_9',
       'POSTAL_CODE_2nd_digit_1', 'POSTAL_CODE_2nd_digit_2',
       'POSTAL_CODE_2nd_digit_3', 'POSTAL_CODE_2nd_digit_4',
       'POSTAL_CODE_2nd_digit_5', 'POSTAL_CODE_2nd_digit_6',
       'POSTAL_CODE_2nd_digit_7', 'POSTAL_CODE_2nd_digit_8',
       'POSTAL_CODE_2nd_digit_9', 'POSTAL_CODE_3rd_d

In [618]:
te.columns

Index(['CREDIT_SCORE', 'ANNUAL_MILEAGE', 'SPEEDING_VIOLATIONS', 'DUIS',
       'PAST_ACCIDENTS', 'AGE_26-39', 'AGE_40-64', 'AGE_65+', 'GENDER_male',
       'DRIVING_EXPERIENCE_10-19y', 'DRIVING_EXPERIENCE_20-29y',
       'DRIVING_EXPERIENCE_30y+', 'EDUCATION_none', 'EDUCATION_university',
       'INCOME_poverty', 'INCOME_upper class', 'INCOME_working class',
       'VEHICLE_OWNERSHIP_1.0', 'VEHICLE_YEAR_before 2015', 'MARRIED_1.0',
       'CHILDREN_1.0', 'TYPE_OF_VEHICLE_SUV', 'TYPE_OF_VEHICLE_Sedan',
       'TYPE_OF_VEHICLE_Sports Car', 'POSTAL_CODE_1st_digit_2',
       'POSTAL_CODE_1st_digit_3', 'POSTAL_CODE_1st_digit_4',
       'POSTAL_CODE_1st_digit_8', 'POSTAL_CODE_1st_digit_9',
       'POSTAL_CODE_2nd_digit_1', 'POSTAL_CODE_2nd_digit_2',
       'POSTAL_CODE_2nd_digit_3', 'POSTAL_CODE_2nd_digit_4',
       'POSTAL_CODE_2nd_digit_5', 'POSTAL_CODE_2nd_digit_6',
       'POSTAL_CODE_2nd_digit_7', 'POSTAL_CODE_2nd_digit_8',
       'POSTAL_CODE_2nd_digit_9', 'POSTAL_CODE_3rd_digit_1',
  

In [619]:
train = tr.copy()
test = te.copy()

In [620]:
# tr= tr.drop(['OUTCOME'], axis = 1).columns

# for i in range(len(tr)):
#     for j in range(i+1, len(tr)):
#         train[tr[i] + '/' + tr[j]] = train[tr[i]]/(1+train[tr[j]])
#         train[tr[i] + '*' + tr[j]] = train[tr[i]]*(train[tr[j]])
#         train[tr[i] + '+' + tr[j]] = train[tr[i]]+train[tr[j]]
#         train[tr[i] + '-' + tr[j]] = train[tr[i]]-train[tr[j]]
        
#         test[tr[i] + '/' + tr[j]] = test[tr[i]]/(1+test[tr[j]])
#         test[tr[i] + '*' + tr[j]] = test[tr[i]]*(test[tr[j]])
#         test[tr[i] + '+' + tr[j]] = test[tr[i]]+test[tr[j]]
#         test[tr[i] + '-' + tr[j]] = test[tr[i]]-test[tr[j]]

In [621]:
X = train.drop(['OUTCOME'], axis = 1)
y = train['OUTCOME']

def generate_feature_scores_df(X,Score):
    feature_score=pd.DataFrame()
    for i in range(X.shape[1]):
        new =pd.DataFrame({"Features":X.columns[i],"Score":Score[i]},index=[i])
        feature_score=pd.concat([feature_score,new])
    return feature_score

def get_selected_features(raw_df,processed_df):
    selected_features=[]
    for i in range(len(processed_df.columns)):
        for j in range(len(raw_df.columns)):
            if (processed_df.iloc[:,i].equals(raw_df.iloc[:,j])):
                selected_features.append(raw_df.columns[j])
    return selected_features
# selector = SelectFromModel(estimator=Lasso(random_state = 42)).fit(X, y)
# fs = selector.get_feature_names_out(X.columns.to_list())
# print(selector.get_support())
# print(fs)


# X_new = SelectKBest(chi2, k=20).fit(X, y)
# feature_score_df= generate_feature_scores_df(X, X_new.scores_)
# print(feature_score_df)

# X_new = X_new.transform(X)
# X_new=pd.DataFrame(X_new)
# selected_features=get_selected_features(X, X_new)#

# # return selected_features
# print(selected_features)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

# # KFold(n_splits=5,shuffle=True,random_state=42)
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)
# pca = PCA(n_components=10)
# X_train = pca.fit_transform(X_train)
# X_test = pca.transform(X_test)

In [622]:
# from sklearn.model_selection import RepeatedStratifiedKFold
# from sklearn.model_selection import GridSearchCV
# model = LogisticRegression()
# solvers = ['newton-cg', 'lbfgs', 'liblinear']
# penalty = ['l2']
# c_values = [100, 10, 1.0, 0.1, 0.01]
# # define grid search
# grid = dict(solver=solvers,penalty=penalty,C=c_values)
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
# grid_result = grid_search.fit(X, y)
# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']

In [623]:
X

,CREDIT_SCORE,ANNUAL_MILEAGE,SPEEDING_VIOLATIONS,DUIS,PAST_ACCIDENTS,AGE_26-39,AGE_40-64,AGE_65+,GENDER_male,DRIVING_EXPERIENCE_10-19y,...,POSTAL_CODE_4th_digit_9,POSTAL_CODE_5th_digit_1,POSTAL_CODE_5th_digit_2,POSTAL_CODE_5th_digit_3,POSTAL_CODE_5th_digit_4,POSTAL_CODE_5th_digit_5,POSTAL_CODE_5th_digit_6,POSTAL_CODE_5th_digit_7,POSTAL_CODE_5th_digit_8,POSTAL_CODE_5th_digit_9
0,0.638050,11000.0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0.475741,9000.0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,0.839817,12000.0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,0.682527,6000.0,1,0,0,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0,1
4,0.572184,15000.0,0,0,1,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104995,0.672255,9000.0,2,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
104996,0.514193,10000.0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
104997,0.461942,11000.0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
104998,0.483571,10000.0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


{'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}

, n_jobs = -1, C = 100, solver = 'newton-cg'

In [624]:
# logModel = LogisticRegression()
# param_grid = [    
#     {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
#     'C' : np.logspace(-4, 4, 10),
#     'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
#     'max_iter' : [1000]
#     }
# ]
# # X_train, X_val, y_train, y_val = train_test_split(
# #     X, y, random_state=42
# # )
# clf = GridSearchCV(logModel, param_grid = param_grid, cv = 3, n_jobs=-1)
# best_clf = clf.fit(X,y)
# best_clf.best_estimator_

C=0.046415888336127774, max_iter=1000, solver='newton-cg'

In [625]:
heh

NameError: name 'heh' is not defined

In [626]:

skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
# skf = KFold(n_splits = 10, shuffle = True, random_state = 42)


cv_score = []
pred_test_full = []
i = 1
# X = X[selected_features]
# test = test[selected_features]
for train_index, test_index in skf.split(X, y):
    print("{} of KFold {}".format(i, skf.n_splits))
    xtr,xvl = X.loc[train_index],X.loc[test_index]
    ytr,yvl = y.loc[train_index],y.loc[test_index]
    sc = StandardScaler()
    nc = MinMaxScaler()
    xt = sc.fit_transform(xtr)
    xv = sc.transform(xvl)
    t = sc.transform(test)
    xtr = pd.DataFrame(nc.fit_transform(xt), columns = xtr.columns)
    xvl = pd.DataFrame(nc.transform(xv), columns = xvl.columns)
    te_df = pd.DataFrame(nc.transform(t), columns = test.columns)
#     xtr = pca.fit_transform(xtr)
#     xvl = pca.transform(xvl)
#     te_df = pca.transform(te_df)
#     model = ExtraTreesRegressor(n_jobs = -1, random_state = 42)
#     model = RandomForestClassifier(n_jobs = -1, random_state = 42)
#     model = XGBClassifier(random_state = 42)
#     model = XGBRegressor(random_state = 42)
#     model = RandomForestRegressor(n_jobs = -1, random_state = 42) 
#     model = ExtraTreesClassifier(n_jobs = -1, random_state = 42)
    model = LogisticRegression(random_state = 42, n_jobs = -1,C=0.046415888336127774, max_iter=1000, solver='newton-cg')
#     model = KNeighborsClassifier(n_neighbors = 25) 
#     model = LGBMClassifier(n_jobs = -1, random_state = 42)
#     model = MultinomialNB()
    model.fit(xtr, ytr)
#     score = log_loss(yvl, model.predict(xvl))
    score = log_loss(yvl, model.predict_proba(xvl)[:, 1])
    print("Log Loss = ", score)
    cv_score.append(score)
    i += 1
    pred_test = model.predict_proba(te_df)[:,1]
#     pred_test = model.predict(te_df)
    pred_test_full.append(pred_test)

1 of KFold 10
Log Loss =  0.681524699247759
2 of KFold 10
Log Loss =  0.6814437736101793
3 of KFold 10
Log Loss =  0.6811911167751062
4 of KFold 10
Log Loss =  0.6814882915991607
5 of KFold 10
Log Loss =  0.6814079651093179
6 of KFold 10
Log Loss =  0.6813200368678967
7 of KFold 10
Log Loss =  0.6815044369111387
8 of KFold 10
Log Loss =  0.6811923181654176
9 of KFold 10
Log Loss =  0.6810319365058112
10 of KFold 10
Log Loss =  0.6808647258970034


In [627]:
print("MEAN CV = ",np.mean(cv_score))
print("Deviation = ", np.std(cv_score))
print("Final local CV = ", np.mean(cv_score) + np.std(cv_score))
print("Final PUBLIC CV = ", np.mean(cv_score) + np.std(cv_score) + 0.04833)

MEAN CV =  0.6812969300688791
Deviation =  0.0002109845946581522
Final local CV =  0.6815079146635373
Final PUBLIC CV =  0.7298379146635373


MEAN CV =  0.6810916386813421
Deviation =  0.00022201759149823776
Final local CV =  0.6813136562728404
Final PUBLIC CV =  0.7296436562728403

In [628]:
med_preds = np.median(pred_test_full, axis = 0)
mean_preds = np.mean(pred_test_full, axis = 0)

In [629]:
hehe

NameError: name 'hehe' is not defined

In [630]:
sample['OUTCOME'] = mean_preds
sample.to_csv('lr_final_tuned_mean.csv', index = False)

In [631]:
sample['OUTCOME'] = med_preds
sample.to_csv('lr_final_tuned_median.csv', index = False)